In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rd
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I

In [2]:
m = ModelIntv()
n_collect = 200

In [3]:
out_path = f'out/dy_hi'
pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

In [4]:
for p in pars_post:
    p.update(r_acf)

In [5]:
results = []


for i, pars in enumerate(pars_post[:n_collect]):
    y1, pars = m.find_baseline(pars, 2022)

    _, ms00, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 0}, 'MU': {'Scale': 0}}, t_end=2031)
    _, ms01, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 0}, 'MU': {'Scale': 1}}, t_end=2031)
    _, ms10, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 1}, 'MU': {'Scale': 0}}, t_end=2031)
    _, ms11, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 1}, 'MU': {'Scale': 1}}, t_end=2031)
    
    inc00 = (ms00.IncR * ms00.Pop).sum() * 0.5
    inc01 = (ms01.IncR * ms01.Pop).sum() * 0.5
    inc10 = (ms10.IncR * ms10.Pop).sum() * 0.5
    inc11 = (ms11.IncR * ms11.Pop).sum() * 0.5
    
    results.append({
        'Key': i,
        'Avt_MU': 1 - inc01 / inc00,
        'Avt_D2D': 1 - inc10 / inc00,
        'Avt_MU+D2D': 1 - inc11 / inc00,
    })
    
results = pd.DataFrame(results)

In [6]:
results.describe()

,Key,Avt_MU,Avt_D2D,Avt_MU+D2D
count,200.000000,200.000000,200.000000,200.000000
mean,99.500000,0.003188,0.014148,0.017228
std,57.879185,0.000323,0.003681,0.003911
min,0.000000,0.002433,0.005023,0.007508
25%,49.750000,0.002986,0.011702,0.014671
50%,99.500000,0.003170,0.014130,0.017184
75%,149.250000,0.003439,0.016590,0.019761
max,199.000000,0.003947,0.023982,0.027693


In [7]:
results.to_csv(f'{out_path}/Sim_BgACF_IncrementalAverted.csv')

In [8]:
idx = ['Pop', 'IncR', 'MorR',
       'ACF_MDU_Footfall', 'ACF_MDU_Screened', 'ACF_MDU_Confirmed', 'ACF_MDU_Yield', 'ACF_MDU_TP',
       'ACF_MDU_DS_Fl', 'ACF_MDU_DR_Fl', 'ACF_MDU_DR_Sl', 
       'ACF_D2D_Footfall', 'ACF_D2D_Screened', 'ACF_D2D_Confirmed', 'ACF_D2D_Yield', 'ACF_D2D_TP',
       'ACF_D2D_DS_Fl', 'ACF_D2D_DR_Fl', 'ACF_D2D_DR_Sl']

results = []


for i, pars in enumerate(pars_post[:n_collect]):
    y1, pars = m.find_baseline(pars, 2022)

    _, ms0, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 0}, 'MU': {'Scale': 0}}, t_end=2031)
    _, ms1, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 1}, 'MU': {'Scale': 1}}, t_end=2031)
    _, ms2, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 2}, 'MU': {'Scale': 2}}, t_end=2031)
    _, ms4, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 4}, 'MU': {'Scale': 4}}, t_end=2031)

    results.append(pd.concat([
        ms0[idx].reset_index().assign(Key=i, Scale='Baseline'),
        ms1[idx].reset_index().assign(Key=i, Scale='1x'),
        ms2[idx].reset_index().assign(Key=i, Scale='2x'),
        ms4[idx].reset_index().assign(Key=i, Scale='4x')
    ]))

results = pd.concat(results)

In [9]:
results.head()

,Time,Pop,IncR,MorR,ACF_MDU_Footfall,ACF_MDU_Screened,ACF_MDU_Confirmed,ACF_MDU_Yield,ACF_MDU_TP,ACF_MDU_DS_Fl,...,ACF_D2D_Footfall,ACF_D2D_Screened,ACF_D2D_Confirmed,ACF_D2D_Yield,ACF_D2D_TP,ACF_D2D_DS_Fl,ACF_D2D_DR_Fl,ACF_D2D_DR_Sl,Key,Scale
0,2022.0,2.383559,0.004928,0.000725,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Baseline
1,2022.5,2.403545,0.004905,0.000721,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Baseline
2,2023.0,2.423699,0.004883,0.000718,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Baseline
3,2023.5,2.444021,0.004861,0.000714,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Baseline
4,2024.0,2.464514,0.004841,0.000711,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Baseline


In [10]:
results.to_csv(f'{out_path}/Sim_BgACF_ScaleUp.csv')